In [1]:
import mysql.connector as mysql
from mysql.connector import Error
import pandas as pd

In [2]:
## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
db = mysql.connect(
    host = "localhost",
    user = "root ",
   passwd = "Giuliagiulia3"
)
print(db)

In [3]:
dataset = pd.read_csv("Airplane_Crashes_and_Fatalities_Since_1908.csv")